# Best Grades 
#### Project by Ran Benayoun

## Project Description

This project scrapes the histogram database from Cheesefork's GitHub repository.

It enables analysis of:
- The **average median grade** for each course
- The **maximum grade** recorded in each course

## Limitations

- The statistics are based only on **available histograms**. Courses with no histograms (typically those with under 10 students or no shared data) are **not included** in the analysis.
- **Course numbers change occasionally**, which can result in data loss or duplication.
- In **2024**, the Technion changed course numbers from 6 digits to 7 or 8 digits. This created inconsistencies in the dataset.


In [2]:
# All the imports needed for the script to function properly
import re
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import requests
from tqdm import tqdm
from time import sleep
import csv

In [ ]:
#Defing functions needed for the script

def find_6_digit_sequences(input_string):
    """
    for catalogs older than 2024
    Finds all unique 6-digit sequences in a given string.

    Args:
        input_string (str): The string to search for 6-digit sequences.

    Returns:
        list: A list of unique 6-digit sequences found in the input string.
    """
    # Define the regex pattern to match 6-digit sequences
    pattern = r'\b\d{6}\b'
    
    # Find all matches using findall() method
    sequences = re.findall(pattern, input_string)
    
    return sequences

def find_7_digit_sequences(input_string):
    """
    for catalogs newer than 2024
    Finds all unique 7-digit sequences in a given string.

    Args:
        input_string (str): The string to search for 76-digit sequences.

    Returns:
        list: A list of unique 7-digit sequences found in the input string.
    """
    # Define the regex pattern to match 7-digit sequences
    pattern = r'\b\d{7}\b'
    
    # Find all matches using findall() method
    sequences = re.findall(pattern, input_string)
    
    return sequences

def clean_string(input_string):
    """
    Cleans a string by removing double quotes, single quotes, new lines, 
    carriage returns, and excessive spaces.

    Args:
        input_string (str): The string to be cleaned.

    Returns:
        str: The cleaned string.
    """
    # Replace unwanted characters
    cleaned_string = input_string.replace('"', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    
    # Remove leading and trailing whitespace
    cleaned_string = cleaned_string.strip()
    
    # Remove excessive spaces between words
    cleaned_string = ' '.join(cleaned_string.split())
    
    return cleaned_string

# Loading the updated catalogs to verify
def load_catalog_file(filename):
    """
    Loads a JSON file and returns the catalog data as a dictionary.

    Args:
        filename (str): The name of the JSON file to be loaded.

    Returns:
        dict: A dictionary containing the catalog data.
    """
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            catalogs = json.load(f)
        return catalogs
    else:
        print(f"File '{filename}' not found.")
        return {}

def plot_bar_chart_with_trendline(average_medians, save_path=None):
    """
    Plots a bar chart of the average medians with a trendline.
    
    Args:
        average_medians (dict): Dictionary with course numbers as keys and average median values as values.
        save_path (str, optional): If provided, saves the figure to the given path.
    """
    # Filter out None values
    courses = [course for course, median in average_medians.items() if median is not None]
    medians = [median for median in average_medians.values() if median is not None]

    plt.figure(figsize=(10, 6))
    plt.bar(courses, medians, color='skyblue', label='Average Median')

    # Add trendline
    x = np.arange(len(courses))
    z = np.polyfit(x, medians, 1)
    p = np.poly1d(z)
    plt.plot(courses, p(x), "r--", label='Trendline')

    plt.title('Bar Plot of Average Medians by Course Number')
    plt.xlabel('Course Number')
    plt.ylabel('Average Median')
    plt.xticks(rotation=45)
    plt.grid(axis='y')
    plt.legend()
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"Figure saved to {save_path}")

    plt.show()

Adding catalog entries with points through the scripts:

In [28]:
import os
import json
import re
import ipywidgets as widgets
from IPython.display import display

def add_to_catalog_file(filename):
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            existing_catalogs = json.load(f)
    else:
        existing_catalogs = {}

    # Catalog name input
    catalog_name_widget = widgets.Text(description='Catalog name:')
    display(catalog_name_widget)

    # Textarea for multiline input
    text_area = widgets.Textarea(
        value='',
        placeholder='Paste course data here...',
        description='Courses:',
        layout=widgets.Layout(width='100%', height='300px')
    )
    display(text_area)

    # Button to process and save
    save_button = widgets.Button(description="Save Catalog")
    output = widgets.Output()
    display(save_button, output)

    def on_button_click(b):
        catalog_name = catalog_name_widget.value.strip()
        input_text = text_area.value
        lines = input_text.strip().split('\n')

        current_catalog_data = {}
        for line in lines:
            line = line.strip()
            if not line:
                continue
            match = re.search(r'(\d+).*?(\d+\.\d+)$', line)
            if match:
                course_number = '0' + match.group(1)
                try:
                    points = float(match.group(2))
                    current_catalog_data[course_number] = points
                except ValueError:
                    print(f"Warning: Could not parse points in: '{line}'")
            else:
                print(f"Warning: Format not recognized: '{line}'")

        existing_catalogs[catalog_name] = current_catalog_data

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(existing_catalogs, f, indent=4, ensure_ascii=False)

        with output:
            output.clear_output()
            print(f"Catalog '{catalog_name}' saved to {filename}")
            print(current_catalog_data)

    save_button.on_click(on_button_click)

add_to_catalog_file("Catalogs_points.json")


Text(value='', description='Catalog name:')

Textarea(value='', description='Courses:', layout=Layout(height='300px', width='100%'), placeholder='Paste cou…

Button(description='Save Catalog', style=ButtonStyle())

Output()

This code goes through the list of courses and extract all the stats from Cheesfork's GitHub while showing a progress bar - got to be patient

In [ ]:
def get_finals_statistics(course_numbers, access_token):
    """
    Fetches the finals statistics for the given course numbers from GitHub.

    Args:
        course_numbers (list of str): List of course numbers to fetch statistics for.
        access_token (str): GitHub access token for authentication.

    Returns:
        dict: A dictionary containing the statistics for each course.
    """
    base_url = "https://api.github.com/repos/michael-maltsev/technion-histograms/contents/"
    headers = {'Authorization': f'token {access_token}'}
    all_statistics = {}

    for course_number in tqdm(course_numbers):
        course_url = f"{base_url}{course_number}?ref=main"
        try:
            response = requests.get(course_url, headers=headers)
            response.raise_for_status()
            contents = response.json()

            course_data = {'folders': {}}

            for item in contents:
                if item['type'] == 'dir':
                    folder_name = item['name']
                    finals_json_url = f"https://raw.githubusercontent.com/michael-maltsev/technion-histograms/main/{course_number}/{folder_name}/Finals.json"
                    finals_response = requests.head(finals_json_url)
                    if finals_response.status_code == 200:
                        finals_data = requests.get(finals_json_url).json()
                        course_data['folders'][folder_name] = {
                            'average': finals_data.get('average'),
                            'median': finals_data.get('median')
                        }

            all_statistics[course_number] = course_data
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for course {course_number}: {e}")
        sleep(0.2)
    return all_statistics

def write_to_csv(statistics):
    """
    Writes the finals statistics to a CSV file.

    Args:
        statistics (dict): The statistics data to write to the CSV.
        filename (str): The name of the CSV file to write to.
    """
    filename = input("Enter the filename to save the CSV: ")
    filename = filename + ".csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Course Number', 'Semester', 'Average', 'Median'])
        for course_number, data in statistics.items():
            for folder, stats in data['folders'].items():
                writer.writerow([course_number, folder, stats['average'], stats['median']])
        writer.writerow(['Course Number', 'Average of Medians'])
        #calculate average of medians to put at the end of the CSV
        average_medians = calculate_average_median(statistics)
        for course_number in average_medians:
            writer.writerow([course_number, average_medians[course_number]])
    print(f"Data has been written to {filename}")

def calculate_average_median(statistics):
    """
    Calculates the average of the median values for each course.

    Args:
        statistics (dict): The statistics data containing median values.

    Returns:
        dict: A dictionary with course numbers as keys and their average median values as values.
    """
    average_medians = {}
    for course_number, data in statistics.items():
        medians = []
        for folder in data['folders'].values():
            median_value = folder['median']
            if median_value is not None:
                try:
                    median_value = round(float(median_value), 2)  # Round to 2 decimal places
                    medians.append(median_value)
                except ValueError:
                    # Skip non-numeric median values silently
                    continue
        
        if medians:
            average_median = round(sum(medians) / len(medians), 2)  # Round to 2 decimal places
        else:
            average_median = None
        average_medians[course_number] = average_median

    return average_medians

import json

def calculate_gpa_from_medians(average_medians, catalog_file, catalog_name):
    """
    Calculates GPA using average medians and course credit points.

    Args:
        average_medians (dict): Dictionary with course numbers as keys and average median values.
        catalog_file (str): Path to Catalogs_points.json.
        catalog_name (str): Name of the catalog to use from the file.

    Returns:
        float: Calculated GPA rounded to 2 decimal places.
    """
    try:
        with open(catalog_file, 'r', encoding='utf-8') as f:
            catalogs = json.load(f)
    except (IOError, json.JSONDecodeError) as e:
        print(f"Error reading catalog file: {e}")
        return None

    if catalog_name not in catalogs:
        print(f"Catalog '{catalog_name}' not found in {catalog_file}")
        return None

    catalog_points = catalogs[catalog_name]

    total_weighted = 0.0
    total_points = 0.0

    for course_number, average_median in average_medians.items():
        if average_median is None:
            continue
        if course_number not in catalog_points:
            continue
        points = catalog_points[course_number]
        total_weighted += average_median * points
        total_points += points

    if total_points == 0:
        print("No valid course points found for GPA calculation.")
        return None

    gpa = round(total_weighted / total_points, 2)
    return gpa


Set up your personal access token from GitHub - this next code will test if it's valid. if not you'll be limited to get 60 courses per hour

In [ ]:
access_token = ''  # Replace with your actual access token

# Test the token validity by making a simple API request
test_url = "https://api.github.com/user"
test_response = requests.get(test_url, headers={'Authorization': f'token {access_token}'})

if test_response.status_code == 200:
    print("Token is valid.")
else:
    print(f"Invalid token. Status code: {test_response.status_code}")
    print(test_response.json())

Token is valid.


# OR 
add directly a string to the catalog file, and the entry will parse through functions to extract the course numbers

In [ ]:
#loaging the updated catalogs to verify
#catalog = load_catalog_file("catalogs.json")

View the content of the catalogs:

In [32]:
#for catalog_name, catalog_content in catalog.items():
#    print(f"{catalog_name}: {catalog_content}")

Load the BME mandatory courses and add a leading "0" for formatting according to Cheesfork data

In [ ]:
# hova_2024 = find_7_digit_sequences(clean_string(catalog['hova_2024']))
# hova_2024 = ["0" + s for s in hova_2024]
# print(hova_2024)


['00440102', '01040065', '01040042', '01140071', '01250001', '01340058', '03340021', '01040042', '01040018', '01040065', '01040016', '01040013', '01040038', '01040136', '01140052', '01240801', '02340128', '00440105', '01040214', '01040215', '01040220', '01240503', '01340019', '03340274', '03240033', '00440131', '01040034', '03340221', '03340222', '03360537', '01340113', '03340009', '03340011', '03350010', '03360022', '03360100', '03340023', '03350001', '03340014', '03350002', '03360026', '03350015', '03350003']


In [ ]:
# #Helps to know what changed each year
# def compare_lists(list1, list2):
#     differences = {
#         "only_in_list1": [item for item in list1 if item not in list2],
#         "only_in_list2": [item for item in list2 if item not in list1],
#         "in_both": [item for item in list1 if item in list2]
#     }
#     return differences

# hova2018 = find_6_digit_sequences(clean_string(catalog['hova_2018']))
# hova2019 = find_6_digit_sequences(clean_string(catalog['hova_2019']))
# hova2020 = find_6_digit_sequences(clean_string(catalog['hova_2020']))
# hova2021 = find_6_digit_sequences(clean_string(catalog['hova_2021']))
# hova2022 = find_6_digit_sequences(clean_string(catalog['hova_2022']))
# hova2023 = find_6_digit_sequences(clean_string(catalog['hova_2023']))

# #compare_lists(hova2019, hova2023)

In [ ]:
# #Setting up "Megamot" course lists for core courses (have to pick at least 2) and the rest of the courses.
# #This list fits the catalog in 2023. any catalog after 2024 includes "Biophysics" and have very different lists.

# #loading biomechanics courses from the catalog
# mech = find_6_digit_sequences(clean_string(catalog['biomechanics']))
# #splitting into mandetory and electives - Biomechanics
# mech_core = mech[:6]
# mech_rest = mech[6:]

# #Setting up signal processing course lists for core courses (have to pick at least 2) and the rest of the courses.
# sig_core = ["336020","336023","336027","336208","336326","336502","336522","336533"]
# sig_rest =["336325","336504","336521","336536","336538","336540","336544","336545","336546","336547","336549","336550","034033","044101","044198","044202","046195","046197","046200","046201","046211","046332","046745","046746","236523","236781"]
# #splitting into mandetory and electives - Tissue engineering
# rekamot = find_6_digit_sequences(clean_string(catalog['Tissue']))
# tissue_core = rekamot[:5]
# tissue_rest = rekamot[5:]

In [ ]:
# hova2024 = get_finals_statistics(hova_2024,access_token)
# averages = calculate_average_median(hova2024)
# print(averages)
# plot_bar_chart_with_trendline(calculate_average_median(hova2024), "average_medians_plot_2024.png")

  0%|          | 0/43 [00:00<?, ?it/s]

 84%|████████▎ | 36/43 [01:54<00:11,  1.66s/it]

Error fetching data for course 03360100: 404 Client Error: Not Found for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/03360100?ref=main


100%|██████████| 43/43 [02:07<00:00,  2.97s/it]


the statistics does not include "Physiology" course (changed number, no histogram)

In [ ]:
# print("Processing Imaging courses:")
# imaging_course_numbers = list(imaging.keys())
# imaging = get_finals_statistics(imaging_course_numbers, access_token)
# ave_imaging = calculate_average_median(imaging)

# print("\nProcessing Biomechanics courses:")
# biomechanics_course_numbers = list(biomechanics.keys())
# biomech = get_finals_statistics(biomechanics_course_numbers, access_token)
# ave_biomech = calculate_average_median(biomech)

# print("\nProcessing Biophysics courses:")
# biophysics_course_numbers = list(biophysics.keys())
# biophys = get_finals_statistics(biophysics_course_numbers, access_token)
# ave_biophys = calculate_average_median(biophys)

# print("\nProcessing Tissue courses:")
# tissue_course_numbers = list(tissue.keys())
# tissue = get_finals_statistics(tissue_course_numbers, access_token)
# ave_tissue = calculate_average_median(tissue)

Processing Imaging courses:


 23%|██▎       | 8/35 [00:09<00:25,  1.04it/s]

Error fetching data for course 03360209: 404 Client Error: Not Found for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/03360209?ref=main


100%|██████████| 35/35 [01:06<00:00,  1.90s/it]



Processing Biomechanics courses:


...

100%|██████████| 30/30 [00:28<00:00,  1.07it/s]



Processing Tissue courses:


 75%|███████▌  | 18/24 [00:13<00:03,  1.53it/s]

Error fetching data for course 00660121: 404 Client Error: Not Found for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/00660121?ref=main


100%|██████████| 24/24 [00:26<00:00,  1.09s/it]


In [ ]:
#ave_tissue

{'03360021': 86.33,
 '03360402': 90.67,
 '03360517': 87.22,
 '03360528': 86.79,
 '03360529': 88.19,
 '03360326': 89.33,
 '03360404': None,
 '03360405': 90.25,
 '03360520': 89.5,
 '03360521': None,
 '03360538': 92.79,
 '03360544': 92.92,
 '03360548': None,
 '03360549': 94.0,
 '03360550': 95.5,
 '00540413': 88.4,
 '00660333': 90.0,
 '01340020': 79.3,
 '01340069': 88.0,
 '01340082': 84.45,
 '01340119': 83.67,
 '01340121': 87.78,
 '02760413': 81.25}

In [25]:
points = {'00440102': None,
 '01040065': 5.0,
 '01040042': 5.0,
 '01140071': 3.5,
 '01250001': 3.0,
 '01340058': 3.0,
 '03340021': 1.0,
 '01040018': 5.0,
 '01040016': 5.0,
 '01040013': 5.5,
 '01040038': 2.5,
 '01040136': 4.0,
 '01140052': 3.5,
 '01240801': 2.5,
 '02340128': 4.0,
 '00440105': 4.0,
 '01040214': 2.5,
 '01040215': 2.5,
 '01040220': 2.5,
 '01240503': 2.5,
 '01340019': 2.5,
 '03340274': 2.0,
 '03240033': 3.0,
 '00440131': 5.0,
 '01040034': 3.5,
 '03340221': 2.5,
 '03340222': 4.0,
 '03360537': 3.0,
 '01340113': 3.5,
 '03340009': 4.0,
 '03340011': 4.0,
 '03350010': 3.0,
 '03360022': 2.5,
 '03340023': 3.5,
 '03350001': 3.5,
 '03340014': 2.0,
 '03350002': 4.0,
 '03360026': None,
 '03350015': 3.0,
 '03350003': 2.0}

In [26]:
# Course numbers to be dropped from the averages calculation
courses_to_drop = ['03240033', '01040018', '01040016']

# Initialize variables for weighted sum and total points
weighted_sum = 0.0
total_points = 0.0

# Iterate through the averages dictionary
for course_number, average in averages.items():
    # Check if the course number should be dropped or if its average is None
    if course_number in courses_to_drop or average is None:
        continue

    # Ensure the course number is a string for consistent key lookup
    course_number_str = str(course_number)

    # Check if the course number exists in the points dictionary
    if course_number_str in points:
        course_points = points[course_number_str]

        # Only include courses with non-zero points in the calculation
        if course_points > 0:
            weighted_sum += (average * course_points)
            total_points += course_points

# Calculate the final weighted average
final_weighted_average = 0.0
if total_points > 0:
    final_weighted_average = weighted_sum / total_points

print(f"The final calculated weighted average is: {final_weighted_average:.2f}")


The final calculated weighted average is: 81.82


In [28]:
import re

text_data = """
Biomechanics:
3360326 םירטמרפ ךורעשו םינותנ חותינ 2 1 - 2.5
3360506 תימוקיש הקינכמויב 2 1 - 2.5
3360517 ויב - אתה לש הסדנה 2 1 - 2.5
3360522 ויב תוכרעמב הרקבל אובמ - תויאופר 2 2 - 3.0
3360539 המישנה תוכרעמב המירז 2 1 - 2.5
3360541 תירלוקסווידרקה תכרעמב המירז 2 1 - 2.5
3360021 וננ - יגולואירו הקינכמ 'לויבב םיקיקלח 2 1 - 2.5
3360520 המקר יפילחתו םיידפוטרוא םילתש 2 1 - 2.5
3360521 וידרקה 'עמ לש םייסדנה תונורקע 3 1 - 3.5
3360540 בשחוממ יאופר רושכמ ןכת 2 1 - 2.5
3360549 ףוציר תוקינכט DNA 2 2 - 3.0
3360550 תיבושיח הקיזיפויב 2 1 - 2.5
0340033 'מ תירמונ הזילנא 2 2 - 3.0
0350001 הקיטובורל אובמ 2 1 - 2.5
0350199 המירזה תרותב בשחמה שומיש 2 2 - 3.0
0360049 הקיטסונגאידו הרקבל תויבצע תותשר 2 1 - 2.5
0360072 הקיטובורו הקינכמויבב הקיטמטניק 3 - - 3.0
0360076 וננב הקיטניק ורטקלא - ורקימו - המירז 3 - - 3.0
0360086 נורקימ םינקתהב רבעמ תועפותו המירז 2 1 1 3.0
0360090 םיאת לש תינכמ השיחו הקינכמויב 3 -- - 3.0
0840225 הקימניד 3 2 - 4.0
2360332 םירבדה לש טנרטניאה - תויגולונכט 2 - - 2.0
2360333 םירבדה לש טנרטניאב טקיורפ 2 - 4 3.0
"""

# Current category being processed
category_name = input("Enter the category name for the first course: ")
# Create a dictionary with the category name as the variable name
globals()[category_name] = {}
current_category = globals()[category_name]
# Split the text into lines
lines = text_data.strip().split('\n')

for line in lines:
    line = line.strip()
    # Skip empty lines
    if not line:
        continue

    # Regex to find the course number (first group of digits) and the points (last float)
    match = re.search(r'(\d+).*?(\d+\.\d+)$', line)
    if match:
        course_number = match.group(1)
        points_str = match.group(2)
        course_number = '0' + course_number

        try:
            points = float(points_str)
            current_category[course_number] = points
        except ValueError:
            print(f"Warning: Could not parse points for line: {line}")
    else:
        # This handles cases where a line might not contain a course number and points
        # but isn't a category header (e.g., empty lines within a category, or other descriptive text)
        pass # Silently ignore lines that don't match the pattern

print(f"{category_name} =", current_category)


biomech = {'03360326': 2.5, '03360506': 2.5, '03360517': 2.5, '03360522': 3.0, '03360539': 2.5, '03360541': 2.5, '03360021': 2.5, '03360520': 2.5, '03360521': 3.5, '03360540': 2.5, '03360549': 3.0, '03360550': 2.5, '00340033': 3.0, '00350001': 2.5, '00350199': 3.0, '00360049': 2.5, '00360072': 3.0, '00360076': 3.0, '00360086': 3.0, '00360090': 3.0, '00840225': 4.0, '02360332': 2.0, '02360333': 3.0}


We can't conclude much from that stats, but we can debate about that :)

Now i'm gonna find out which course had an exam that no one could ace

In [13]:
def get_max_statistics(course_numbers, access_token):
    """
    Fetches the maximum grade statistics for the given course numbers from GitHub,
    considering "Exam_A.json" and "Exam_B.json", excluding grades of 100.

    Args:
        course_numbers (list of str): List of course numbers to fetch statistics for.
        access_token (str): GitHub access token for authentication.

    Returns:
        dict: A dictionary containing the maximum grade statistics for each course.
    """
    statistics = {}
    base_url = "https://api.github.com/repos/michael-maltsev/technion-histograms/contents/"
    headers = {'Authorization': f'token {access_token}'}

    for course_number in tqdm(course_numbers):
        course_url = f"{base_url}{course_number}?ref=main"
        try:
            response = requests.get(course_url, headers=headers)
            response.raise_for_status()
            folders = response.json()

            course_statistics = {"Exam_A": {}, "Exam_B": {}}
            for folder in folders:
                if folder['type'] == 'dir':
                    folder_name = folder['name']
                    for exam in ["Exam_A.json", "Exam_B.json"]:
                        exam_key = exam.split('.')[0]  # Either "Exam_A" or "Exam_B"
                        exam_url =f"https://raw.githubusercontent.com/michael-maltsev/technion-histograms/main/{course_number}/{folder_name}/{exam}"
                        response = requests.get(exam_url, headers=headers)
                        if response.status_code == 200:
                            exam_data = response.json()
                            max_grade = exam_data.get("max")
                            if max_grade and max_grade != 100:
                                if folder_name not in course_statistics[exam_key]:
                                    course_statistics[exam_key][folder_name] = max_grade
                                else:
                                    course_statistics[exam_key][folder_name] = max(max_grade, course_statistics[exam_key][folder_name])
            statistics[course_number] = course_statistics
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for course {course_number}: {e}")
    sleep(0.2)
    return statistics

def write_max_statistics_to_csv(statistics, filename):
    """
    Writes the maximum grade statistics to a CSV file.

    Args:
        statistics (dict): A dictionary containing the maximum grade statistics for each course.
        filename (str): The name of the CSV file to be created.
    """
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Course Number", "Folder", "Exam", "Max Grade"])

        for course_number, exams in statistics.items():
            # Get all unique folders
            all_folders = set()
            for exam_key, folders in exams.items():
                all_folders.update(folders.keys())
            
            # Write data for each folder
            for folder_name in all_folders:
                for exam_key in ['Exam_A', 'Exam_B']:
                    if exam_key in exams and folder_name in exams[exam_key]:
                        max_grade = exams[exam_key][folder_name]
                        writer.writerow([course_number, folder_name, exam_key, max_grade])


In [ ]:
write_max_statistics_to_csv(get_max_statistics(find_6_digit_sequences(clean_string(catalog['mechonot'])), access_token), "MechMax.csv")

 79%|███████████████████████████████████████████████████████████████▎                | 34/43 [08:53<01:05,  7.24s/it]

Error fetching data for course 034057: 404 Client Error: Not Found for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/034057?ref=main


 91%|████████████████████████████████████████████████████████████████████████▌       | 39/43 [09:22<00:19,  4.93s/it]

Error fetching data for course 034382: 404 Client Error: Not Found for url: https://api.github.com/repos/michael-maltsev/technion-histograms/contents/034382?ref=main


100%|████████████████████████████████████████████████████████████████████████████████| 43/43 [09:39<00:00, 13.47s/it]


<IPython.core.display.Javascript object>